In [ ]:
 import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/AD Identification using SATD'

Mounted at /content/drive
/content/drive/My Drive/AD Identification using SATD


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.colors as mcolors

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, make_scorer

#Read and process the dataset

In [ ]:

liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)

In [ ]:
liu_['Comments'].fillna('', inplace=True)
liu_['TDType'] = liu_['TDType'].astype(str)

values_to_remove = ['MULTITHREAD', 'nan', 'removeType']
replacement_value = 'WITHOUT_CLASSIFICATION'
liu_['TDType'].replace(values_to_remove, replacement_value, inplace=True)

liu_['Comments'] = liu_['Comments'].str.replace('content=', '', regex=False)
liu_['Comments'] = liu_['Comments'].str.replace('"', '', regex=False)

liu_ = liu_.drop_duplicates(subset=['Comments', 'TDType'])

liu_['TDType'] = liu_['TDType'].replace('removeType', 'WITHOUT_CLASSIFICATION')

In [ ]:
# Count the number of duplicate rows in the DataFrame
num_duplicates = liu_.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")


Number of duplicate rows: 0


##RF +TFIDF

In [ ]:
data = liu_

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data['Comments'], data['TDType'], test_size=0.2, random_state=42)
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the pipeline for training
pipeline = Pipeline([
    ('TFIDF', TfidfVectorizer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [None,2, 5, 10],
    'clf__min_samples_leaf': [1, 2],

}


grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)


grid_search.fit(X_train_final, y_train_final)
print("Best parameters found:", grid_search.best_params_)

# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
conf_matrix_val = confusion_matrix(y_val, y_val_pred)
classification_rep_val = classification_report(y_val, y_val_pred)

print("Validation Confusion Matrix:")
print(conf_matrix_val)
print("\nValidation Classification Report:")
print(classification_rep_val)

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classification_rep_test = classification_report(y_test, y_test_pred)

print("\nTest Confusion Matrix:")
print(conf_matrix_test)
print("\nTest Classification Report:")
print(classification_rep_test)


Fitting 5 folds for each of 64 candidates, totalling 320 fits
Best parameters found: {'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 5, 'clf__n_estimators': 200}
Validation Confusion Matrix:
[[  28    0    0   75    0    5    0   35]
 [   0   18    0   33    0    3    0   13]
 [   0    1   32   51    0    2    2   11]
 [   6    2    6 1513    0   54    3  132]
 [   0    0    0    9    9    0    0    2]
 [   1    0    4  127    2  197    1   16]
 [   0    0    2   34    0    2   61    6]
 [   1    1    0   79    0    2    0 3639]]

Validation Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.78      0.20      0.31       143
         COMPATIBILITY       0.82      0.27      0.40        67
                DEFECT       0.73      0.32      0.45        99
                DESIGN       0.79      0.88      0.83      1716
         DOCUMENTATION       0.82      0.45      0.58        20
        IMPLE